
# INSTALLATION

In [1]:
!pip install aif360
!pip install fairlearn

     |████████████████████████████████| 175 kB 4.9 MB/s 
     |████████████████████████████████| 356 kB 39.6 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30190 sha256=41638016d255b27b8cfbb7719617dcffbba29ce16ea66f0de9aec52f68c3100a
  Stored in directory: /root/.cache/pip/wheels/56/19/d5/8cad06661aec65a04a0d6785b1a5ad035cb645b1772a4a0882
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491649 sha256=305b15b26005c4a595c3051a354ff632d555cd835b8a985658a74591f97dbb61
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built memory-profiler shap
     |████████████████████████████████| 177 kB 5.0 MB/s 


In [2]:
!apt-get install -jre
!java -version

E: Command line option 'j' [from -jre] is not understood in combination with the other options.
openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [3]:
!pip install h2o

     |████████████████████████████████| 168.4 MB 38 kB/s 
  Created wheel for h2o: filename=h2o-3.32.1.6-py2.py3-none-any.whl size=168439194 sha256=8d34d3ad338d80f29fb6f71a750afc1973e798bbc9079b81f572b90901064e59
  Stored in directory: /root/.cache/pip/wheels/ee/0f/51/849ba221c4c1b11a04efb4a3427dc9cb1c4dcde218c6c98b13
Successfully built h2o


In [4]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 5.0 MB/s 


#IMPORTS

In [5]:
import numpy as np
from mlxtend.feature_selection import  ExhaustiveFeatureSelector
from xgboost import  XGBClassifier
# import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl import  load_workbook

import shap
#suppress setwith copy warning
pd.set_option('mode.chained_assignment',None)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectFwe, SelectPercentile,SelectFdr, SelectFpr, SelectFromModel
from sklearn.feature_selection import chi2, mutual_info_classif
# from skfeature.function.similarity_based import fisher_score
import aif360
import matplotlib.pyplot as plt
from aif360.metrics.classification_metric import ClassificationMetric
from aif360.algorithms.postprocessing import EqOddsPostprocessing ,RejectOptionClassification,CalibratedEqOddsPostprocessing

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

import sys
sys.path.append("../")
import os


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyxj5spkk
  JVM stdout: /tmp/tmpyxj5spkk/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpyxj5spkk/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_unknownUser_ni8gj6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


#**************************LOADING DATASET*******************************

In [8]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [9]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Compas/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Compas/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)

  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  
  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "two_year_recid"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()
  aml = H2OAutoML(max_models=10, nfolds=10, include_algos=['GBM'] , stopping_metric='AUTO') #verbosity='info',,'GBM', 'DRF'
  aml.train(x=x, y=y, training_frame=Train)
  best_model= aml.leader
  # a.model_performance()
  #**********************REPLACE LABELS OF DUPLICATED TRAIN AND  TEST SET WITH 0.5 THRESHOLDED PREDICT PROBA****************************
  #predicted proba for train labels
  gbm_Predictions_train= best_model.predict(Train)
  gbm_Predictions_train= gbm_Predictions_train.as_data_frame()
  train_label= (gbm_Predictions_train.p1>0.5).astype(int)
  predicted_df_train= train.copy()
  predicted_df_train["two_year_recid"]= train_label

  #predicted proba for test labels
  gbm_Predictions_test= best_model.predict(Test)
  gbm_Predictions_test= gbm_Predictions_test.as_data_frame()
  test_label= (gbm_Predictions_test.p1>0.5).astype(int)
  predicted_df_test= test.copy()
  predicted_df_test["two_year_recid"]= test_label


  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************
  '''the EO's fit method takes as input the original data (ground truth) and the predicted dataset with the predicted dataset having 
  same features but different labels (label for predicted dataset is the thresholded predict proba). as TPR and FPR requires \hat{Y} and Y 
  Converting the  two required dataset to binary format as accepted by EO.
  ''' 


  #Transforming  the Ground truth Train Test to BLD
  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class Train(StandardDataset):
      def __init__(self,label_name= "two_year_recid",
                   favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
             )

  BLD_Train= Train(protected_attribute_names= ['race'],
                         privileged_classes= [[1]])

  class Test(StandardDataset):
      def __init__(self,label_name= "two_year_recid",
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #**************************************Predicted Train Test BLD*****************************************
  class PreTrain(StandardDataset):
      def __init__(self,label_name= "two_year_recid",
                   favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PreTrain, self).__init__(df=predicted_df_train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
             )

  BLD_PredictedTrain= PreTrain(protected_attribute_names= ['race'],
                         privileged_classes= [[1]])

  class PreTest(StandardDataset):
      def __init__(self,label_name= "two_year_recid",
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PreTest, self).__init__(df=predicted_df_test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredictedTest= PreTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #****************************************Equal Opportunity Instance*******************************
  Equalizer= CalibratedEqOddsPostprocessing(unprivileged_groups= disadvantagedGroup , privileged_groups= advantagedGroup )
  #uses ground truth and unfair predictions of classifier to determine probabilities with which unfair output labels are changed to satisfy EO
  Equalizer.fit(BLD_Train,BLD_PredictedTrain )
  #predicting the new labels assigned by the EO engine
  BLD_PredictedTest= Equalizer.predict(BLD_PredictedTest)

  # ********************COMPUTE DISCRIMINATION*****************************

  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/CalEqualOdds/CalEO_gbm.xlsx')
  Compas= excelBook['Compas']
  data= Compas.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredictedTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/CalEqualOdds/CalEO_gbm.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Compas', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6677471636952999
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6977309562398704
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |████████████████████████████████████

#LOGISTIC REGRESSION

In [10]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Compas/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path) 

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Compas/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path) 

  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "two_year_recid"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()

  LogReg = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0)
  LogReg.train(x=x, y=y, training_frame=Train)

  #**********************REPLACE LABELS OF DUPLICATED TRAIN AND  TEST SET WITH 0.5 THRESHOLDED PREDICT PROBA****************************
  #predicted proba for train labels
  lr_Predictions_train= LogReg.predict(Train)
  lr_Predictions_train= lr_Predictions_train.as_data_frame()
  train_label= (lr_Predictions_train.p1>0.5).astype(int)
  predicted_df_train= train.copy()
  predicted_df_train["two_year_recid"]= train_label

  #predicted proba for test labels
  lr_Predictions_test= LogReg.predict(Test)
  lr_Predictions_test= lr_Predictions_test.as_data_frame()
  test_label= (lr_Predictions_test.p1>0.5).astype(int)
  predicted_df_test= test.copy()
  predicted_df_test["two_year_recid"]= test_label


  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************
  '''the EO's fit method takes as input the original data (ground truth) and the predicted dataset with the predicted dataset having 
  same features but different labels (label for predicted dataset is the thresholded predict proba). as TPR and FPR requires \hat{Y} and Y 
  Converting the  two required dataset to binary format as accepted by EO.
  ''' 


  #Transforming  the Ground truth Train Test to BLD
  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class Train(StandardDataset):
      def __init__(self,label_name= "two_year_recid",
                   favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
             )

  BLD_Train= Train(protected_attribute_names= ['race'],
                         privileged_classes= [[1]])

  class Test(StandardDataset):
      def __init__(self,label_name= "two_year_recid",
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #**************************************Predicted Train Test BLD*****************************************
  class PreTrain(StandardDataset):
      def __init__(self,label_name= "two_year_recid",
                   favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PreTrain, self).__init__(df=predicted_df_train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
             )

  BLD_PredictedTrain= PreTrain(protected_attribute_names= ['race'],
                         privileged_classes= [[1]])

  class PreTest(StandardDataset):
      def __init__(self,label_name= "two_year_recid",
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PreTest, self).__init__(df=predicted_df_test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredictedTest= PreTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #****************************************Equal Opportunity Instance*******************************
  Equalizer= CalibratedEqOddsPostprocessing(unprivileged_groups= disadvantagedGroup , privileged_groups= advantagedGroup )
  #uses ground truth and unfair predictions of classifier to determine probabilities with which unfair output labels are changed to satisfy EO
  Equalizer.fit(BLD_Train,BLD_PredictedTrain )
  #predicting the new labels based on the fitted EO engine
  BLD_PredictedTest= Equalizer.predict(BLD_PredictedTest)

  # ***************************COMPUTE DISCRIMINATION********************************

  excelBook= load_workbook("/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/CalEqualOdds/CalEO_LogReg.xlsx")
  Compas= excelBook['Compas']
  data= Compas.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredictedTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/CalEqualOdds/CalEO_LogReg.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Compas', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Poss Pyrrolidinobutiophenone, c_charge_desc_Possession Child Pornography, c_charge_desc_Violation Of Boater Safety Id, c_charge_desc_Structuring Transactions, c_charge_desc_DUI - Property Damage/Personal Injury, c_charge_desc_Attempted Deliv Control Subst, c_charge_desc_Stalking, c_charge_desc_Traff In Cocaine <400g>150 Kil, c_charge_desc_Exhibition Weapon School Prop, c_charge_desc_Retail Theft $300 1st Offense, c_charge_desc_Poss/Sell/Del/Man Amobarbital, c_charge_desc_Crlty Twrd Child Urge Oth Act, c_charge_desc_Felon in Pos of Firearm or Amm, c_charge_desc_Intoxicated/Safety Of Another, c_charge_desc_Poss Of 1,4-Butanediol, c_charge_desc_Harm Public Servant Or Family, c_charge_desc_Battery Emergency Care Provide, c_charge_desc_Sell/Man/Del Pos/w/int Heroin, c_charge_desc_Battery Spouse Or Girlfriend, c_charge_desc_Possession Of 3,4Methylenediox, c_charge_desc_Del of JWH-250 2-Methox 1-Pentyl, c_charge_desc_Battery On Fire Fighter, c

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6474878444084279
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Oper Motorcycle W/O Valid DL, c_charge_desc_Robbery W/Deadly Weapon, c_charge_desc_Fail Sex Offend Report Bylaw, c_charge_desc_Tresspass in Structure or Conveyance, c_charge_desc_False 911 Call, c_charge_desc_Solicit To Deliver Cocaine, c_charge_desc_Carrying A Concealed Weapon, c_charge_desc_Trespass Property w/Dang Weap, c_charge_desc_Poss Pyrrolidinovalerophenone W/I/D/S, c_charge_desc_Possess Weapon On School Prop, c_charge_desc_Purchase/P/W/Int Cannabis, c_charge_desc_Sale/Del Counterfeit Cont Subs, c_charge_desc_Use Of 2 Way Device To Fac Fel, c_charge_desc_Agg Abuse Elderlly/Disabled Adult, c_charge_desc_Disrupting School Function, c_charge_desc_Misuse Of 911 Or E911 System, c_charge_desc_Obtain Control Substance By Fraud, c_charge_desc_Possession Of Buprenorphine, c_charge_desc_Possess w/I/Utter Forged Bills, c_charge_desc_Consume Alcoholic Bev Pub, c_charge_desc_Littering, c_charge_desc_Unlawful Conveyance of Fuel, c_charge_des

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6750405186385737
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


...

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6553122465531225
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Attempt Armed Burglary Dwell, c_charge_desc_Unauthorized Interf w/Railroad, c_charge_desc_Violation Of Boater Safety Id, c_charge_desc_Theft/To Deprive, c_charge_desc_Attempted Burg/struct/unocc, c_charge_desc_Tampering with a Victim, c_charge_desc_Trespass Property w/Dang Weap, c_charge_desc_Neglect/Abuse Elderly Person, c_charge_desc_Crlty Twrd Child Urge Oth Act, c_charge_desc_Possession Of Diazepam, c_charge_desc_Possess Weapon On School Prop, c_charge_desc_Sex Batt Faml/Cust Vict 12-17Y, c_charge_desc_Possession Of Phentermine, c_charge_desc_Disrupting School Function, c_charge_desc_Sex Battery Deft 18+/Vict 11-, c_charge_desc_Sound Articles Over 100, c_charge_desc_Poss Counterfeit Payment Inst, c_charge_desc_Possession of Methadone, c_charge_desc_Obtain Control Substance By Fraud, c_charge_desc_Uttering Worthless Check +$150, c_charge_desc_Defrauding Innkeeper $300/More, c_charge_desc_Possession Firearm School Prop, c_charge_desc_

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6528791565287916
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Oper Motorcycle W/O Valid DL, c_charge_desc_Deliver Cocaine 1000FT Church, c_charge_desc_Possession Child Pornography, c_charge_desc_Bribery Athletic Contests, c_charge_desc_Aggravated Battery (Firearm/Actual Possession), c_charge_desc_Hiring with Intent to Defraud, c_charge_desc_Cruelty to Animals, c_charge_desc_Fail Sex Offend Report Bylaw, c_charge_desc_DUI - Property Damage/Personal Injury, c_charge_desc_False 911 Call, c_charge_desc_Traffick Oxycodone     4g><14g, c_charge_desc_False Info LEO During Invest, c_charge_desc_Computer Pornography, c_charge_desc_Lewd/Lasciv Molest Elder Persn, c_charge_desc_Poss Anti-Shoplifting Device, c_charge_desc_Poss Pyrrolidinovalerophenone W/I/D/S, c_charge_desc_Felon in Pos of Firearm or Amm, c_charge_desc_Refuse Submit Blood/Breath Test, c_charge_desc_Intoxicated/Safety Of Another, c_charge_desc_Agg Abuse Elderlly/Disabled Adult, c_charge_desc_Unl/Disturb Education/Instui, c_charge_desc_Burglary

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6366585563665855
